In [73]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [74]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [75]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [76]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [77]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])
readInJson(garbageCollectionDate[3]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_c2f141ec_a3ac_4b42_a397_ba5ed0c5da46.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Thursday - Calendar B', 'rub_desc': ..."
1,Feature,ckan_c2f141ec_a3ac_4b42_a397_ba5ed0c5da46.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,{'name': 'Thursday - Calendar B (inc. Organics...


In [78]:
df = readInJson(garbageCollectionDate[3])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))

df.loc[df.geo_nested == 1,].shape

(0, 7)

In [79]:
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [80]:
df

,properties,lat,long
0,"{'name': 'Thursday - Calendar B', 'rub_desc': ...",144.221659,-36.716765
0,"{'name': 'Thursday - Calendar B', 'rub_desc': ...",144.222379,-36.717348
0,"{'name': 'Thursday - Calendar B', 'rub_desc': ...",144.222931,-36.716829
0,"{'name': 'Thursday - Calendar B', 'rub_desc': ...",144.223166,-36.716607
0,"{'name': 'Thursday - Calendar B', 'rub_desc': ...",144.223854,-36.717071
...,...,...,...
60,"{'name': 'Wednesday - Calendar A', 'rub_desc':...",144.257785,-36.795824
60,"{'name': 'Wednesday - Calendar A', 'rub_desc':...",144.257795,-36.795162
60,"{'name': 'Wednesday - Calendar A', 'rub_desc':...",144.258293,-36.795082
60,"{'name': 'Wednesday - Calendar A', 'rub_desc':...",144.257835,-36.791892


In [81]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''
        
    returningStr = rub_colFreq + rub_colDay
    
    if len(returningStr.split())>1:
         return returningStr
    else: 
        return 'Unavailable'

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

In [82]:
df.drop('properties', axis = 1, inplace = True)

In [83]:
bendigo = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})

bendigo = bendigo.loc[bendigo.collection_time!= 'Unavailable']

bendigo.to_csv('../GBD_bendigo.csv', index = False)